# **Project some statistical results on the freesurfer surfaces**
This notebook will guide you through the step necessary to project fMRI results on the surfaces that were generated with Freesurfer.

In [ ]:
# initiate some paths and files
SUBJ=Danny

# where will the result appear
OUT=/Users/chris/Desktop/Epi2Surf_${SUBJ}
mkdir -p $OUT


EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/final/sub-danny/ses-20180117/func/
EPI=${EPI_ROOT}/sub-danny_ses-20180117_task-prf_run-01_frame-10_bold_res-1x1x1_reference_zcrop.nii.gz
# the functional space the results are in. This is usually the reference from the `manual-masks' folder that is used for preprocessing and modelfit in NHP-BIDS.
EPI_MASK=${EPI_ROOT}/T1_to_func_brainmask_zcrop.nii.gz
# the brain mask. Will likely be present in the same folder as the epi-reference

NOTEBOOK_PATH=/Users/chris/Documents/MRI_ANALYSIS/NHP-Freesurfer/Notebooks

In [ ]:
# Mask the epi that is used as a reference in functional analysis to extract the brain
fslmaths ${EPI} -mas ${EPI_MASK} ${OUT}/epi_brain.nii.gz
EPI_BRAIN=${OUT}/epi_brain.nii.gz

## Flirt registration of epi to T1
The T1 you pick here is the header adjusted one that formed the based of the surface generation in Freesurfer. We do not have to adjust the header of the epi, because flirt will generally be able to take of the scaling. If this fails for some reason you can still do it manually with `3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen <epi>.nii.gz`. Note that we set voxel sizes to 2 mm in the header in this case. The reason is that for the T1 we had 0.5 mm voxels and adjusted the header to 1 mm, a factor of 2. Our pre-processed epi's have 1 mm voxels, so applying the same factor of 2 means the header info should state that the voxels are 2 mm isotropic.

Be aware that, when you first adjust the header and then do flirt, the final registration matrix will not include this scaling and expect header-adjusted input. Any result you want to warp to the surface will thus first have to be adjusted. You don't have to do this is if the flirt is performed on the non-adjusted epi. 

NB1! One thing to carefully check is whether the x-direction of the voxel order matches between the epi and T1 because if it doesn't we will see left/right flips. With some of our older standard epi's this needs correction (later I corrected it in the reference file for pre-processing). There's a script that does that for you called `swap_xdir_voxels.sh`. You can find it in the `bin` folder of the `Process-NHP-MRI` repository (https://github.com/VisionandCognition/Process-NHP-MRI)

NB2! flirt works significantly better if you include the white matter segmentation (`-wmseg`) of the T1 and phase encoding direction (`-pedir`) of the epi. Check the documentation to find how to code this. If you used Chris Klink's standard epi sequence, your `-pedir` is `-2`.

In [ ]:
# $SUBJECTS_DIR should point to the Freesurfer subjects directory. Adjust your bashrc to export this if it doesn't do so already.

# get the brain and white matter volumes from freesurfer & convert to nifti
mri_convert ${SUBJECTS_DIR}/${SUBJ}/mri/brain.mgz ${OUT}/brain.nii.gz
mri_convert ${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz ${OUT}/wm.nii.gz

flirt -ref ${OUT}/brain.nii.gz -wmseg ${OUT}/wm.nii.gz -in ${EPI_BRAIN} -out ${OUT}/epi2anat.nii.gz -omat ${OUT}/epi2anat.mat -pedir -2   

## Create the tkregister matrix
Now we will use the flirt registration matrix to create a registration matrix in freesurfer format. To get there, we can use the freesurfer program `tkregister` that allows manual registration between 2 volumes. We will inititate it with the flirt matrix and check whether registration is good. If not, you can make manual adjustments, but flirt probably does a better job than manual attempts. Within `tkregister` you can check the alignment of the two volumes by clicking `compare`. Save the registration matrix as `reg.fsl.dat`.

![tkregister](pics/tkregister.gif "tkregister")

In [ ]:
tkregister2 --mov ${EPI_BRAIN} --targ ${OUT}/brain.nii.gz --fsl ${OUT}/epi2anat.mat --reg ${OUT}/reg.fsl.dat --s ${SUBJ}

## Convert volumes to surface representation

In [ ]:
# These (re)definitions allow to do this part independent of previous steps
SUBJ=${SUBJ} 
RegMat=${OUT}/reg.fsl.dat 

RES_IN=${NOTEBOOK_PATH}/example_data/tstat1.nii.gz # this is one of the visual localizers in our curve tracing experiment >> left side stimulus look at rh
RES_IN2=${NOTEBOOK_PATH}/example_data/MeanAngle_th4_pv0.2.nii.gz # this is one of the visual localizers in our curve tracing experiment

In [ ]:
mri_vol2surf --help

In [ ]:
# example surface plot
# white matter surface
mri_vol2surf --trgsubject ${SUBJ} --src ${RES_IN} --out ${OUT}/targ1_wm.w --out_type paint --projfrac 0 --srcreg ${OUT}/reg.fsl.dat --hemi lh --hemi rh
# midcortical
mri_vol2surf --trgsubject ${SUBJ} --src ${RES_IN} --out ${OUT}/targ1_midcort.w --out_type paint --projfrac 0.5 --srcreg ${OUT}/reg.fsl.dat --hemi lh --hemi rh
# average on normal
mri_vol2surf --trgsubject ${SUBJ} --src ${RES_IN} --out ${OUT}/targ1_avg.w --out_type paint --projfrac-avg 0 1 0.2 --srcreg ${OUT}/reg.fsl.dat --hemi lh --hemi rh

In [40]:
flirt -ref ${OUT}/brain.nii.gz -in ${RES_IN2} -out ${OUT}/angle.nii.gz -applyxfm -init ${OUT}/epi2anat.mat
mri_convert ${OUT}/angle.nii.gz ${OUT}/angle.mgz

mri_convert.bin /Users/chris/Desktop/Epi2Surf_Danny/angle.nii.gz /Users/chris/Desktop/Epi2Surf_Danny/angle.mgz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /Users/chris/Desktop/Epi2Surf_Danny/angle.nii.gz...
TR=1000.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/Desktop/Epi2Surf_Danny/angle.mgz...


In [ ]:
# show
tksurfer ${SUBJ} rh graymid -patch full.patch.flat -overlay ${OUT}/targ1_avg.w -overlay-reg ${OUT}/reg.fsl.dat

Results will look somewhat like this:
(but these images are from a different example/hemisphere)

![tstat_inflated](pics/tstat_inflated.png "tstat_inflated") ![tstat_inflated](pics/tstat_inflated_legend.png "tstat_inflated_legend")

(Left: full.patch, Right: occip.patch):

![tstat_fullflat](pics/tstat_fullflat.png "tstat_fullflat") ![tstat_occipflat](pics/tstat_occipflat.png "tstat_occipflat")

## Project a retinotopic map on the surface

In [ ]:
mri_vol2surf --trgsubject ${SUBJ} --src ${RES_IN2} --out ${OUT}/angle_avg.w --out_type paint --projfrac-avg 0 1 0.2 --srcreg ${OUT}/reg.fsl.dat --hemi lh

fslmaths ${RES_IN2} -mul 0.01745329252 ${OUT}/angle_rad.nii.gz

mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_rad.nii.gz --out ${OUT}/angle_avg_rad.w --out_type paint --projfrac-avg 0 1 0.2 --srcreg ${OUT}/reg.fsl.dat --hemi lh


In [ ]:
tksurfer ${SUBJ} lh graymid -patch full.patch.flat -overlay ${OUT}/angle_avg_rad.w -overlay-reg ${OUT}/reg.fsl.dat

## Curve tracing results Danny

In [ ]:
ct=(ct-UL ct-DL ct-UR ct-DR)
for R in ${ct[@]}; do
    mri_vol2surf --trgsubject ${SUBJ} --src ${NOTEBOOK_PATH}/example_data/${R}/tstat1.nii.gz \
    --out ${OUT}/${R}.w --out_type paint \
    --projfrac 1 \
    --srcreg ${OUT}/reg.fsl.dat --hemi lh --hemi rh
done

In [ ]:
tksurfer ${SUBJ} rh graymid -patch full.patch.flat \
    -overlay ${OUT}/ct-DL.w -truncphaseflag 1 \
    -overlay-reg ${OUT}/reg.fsl.dat

In [ ]:
tksurfer ${SUBJ} lh graymid -patch full.patch.flat \
    -overlay ${OUT}/ct-UR.w -truncphaseflag 1 \
    -overlay-reg ${OUT}/reg.fsl.dat

In [ ]:
# check left-right flip issue
flirt -ref ${OUT}/brain.nii.gz -in ${RES_IN} -out ${OUT}/tstat_warp.nii.gz -applyxfm -init ${OUT}/epi2anat.mat 